# Solving for the Chebyshev-Based Spectral Derivative at Domain Endpoints

There is a major problem in the expanded, "splintered" expressions for $\frac{d^\nu}{dx^\nu} y(\theta)$: Factors in $x$ multiplying each $y{(\nu)} = \frac{d^\nu}{d\theta^\nu}$ have multiples of $\sqrt{1-x^2}$ in their denominators, making them undefined for $x = \pm 1$. We can solve for the endpoints using a L'Hôpital's rule, but the answers are difficult to derive, especially for higher derivatives.

I opened [Issue #1](https://github.com/pavelkomarov/spectral-derivatives/issues/1) to address the ugliness of this with a programmatic solution. An algorithm to derive the endpoint expressions turns out to be pretty challenging, not so easy to turn into compact code, and unfortunately relies on symbolic differentiation and simplification. As such, it has not been put in to the core library code. However, in case anyone should need to take a derivative beyond the $4^{th}$, this notebook implements a process to find the right expressions.

In [1]:
import sympy as sp
from collections import defaultdict, deque
from numpy.polynomial import Polynomial as poly

In [2]:
nu = 5 # The order of derivative you want to work with

## Using $\theta$ Instead of $x$

$$ \frac{d}{dx} y(\theta) = \frac{d}{d\theta}y(\theta) \cdot \frac{d\theta}{dx}$$

And then subsequent derivatives entail a product rule, which is how terms quickly proliferate. These derivatives are pretty ugly, because $\frac{d\theta}{dx} = \frac{d}{dx}\cos^{-1}(x) = \frac{-1}{\sqrt{1-x^2}}$. However, due to the relationship $x = \cos(\theta)$, $\frac{-1}{\sqrt{1-x^2}} = \frac{-1}{\sin(\theta)}$. To avoid taking the limit of a quantity with multiple related variables, it's somewhat cleaner to put everything in terms of $\theta$ and then apply L'Hôpital's rule by taking derivatives w.r.t. $\theta$ rather than $x$. The two are equivalent; we end up with powers of $\sin(\theta)$ in the denominator, which is also stubbornly 0 at the endpoints ($\theta = 0$ and $\pi$).

## Finding the Denominator

If we carry through with L'Hôpital's rule in variable $\theta$, we see that after each differentation a $\sin(\theta)$ cancels with all the terms in the numerator, just like how when we do it in $x$ a $\sqrt{1-x^2}$ cancels, and we can thereby, over the course of $\nu$ applications of the rule, eliminate the troublesome $\sin(\theta)$s in the denominator.

The denominator starts out $\sin^{2\nu-1}(\theta)$ and gets changed simply as in the following loop:

In [4]:
th = sp.symbols('th')
denom = sp.sin(th)**(2*nu - 1)
for i in range(nu-1):
	denom = sp.diff(denom, th)
	denom = sp.cancel(denom/sp.sin(th))
denom = sp.diff(denom, th).trigsimp()

We can now evaluate at the endpoints and get nonzero values. For even $\nu$ the two endpoints have the same denominator; for odd $\nu$ they are $\pm$ each other.

In [6]:
denom_0 = denom.subs(th, 0)
denom_pi = denom.subs(th, sp.pi)
print(denom_0, denom_pi)

945 -945


## Finding the Numerator Terms

We can begin with the pyramid of terms built up for $\frac{d^\nu}{dx^\nu} y(\theta)$ as evaluated in the package.

In [22]:
# Calculate the polynomials in x necessary for transforming back to the Chebyshev domain
numers = deque([poly([-1])]) # just -1 to start, at order 1
denom = poly([1, 0, -1]) # 1 - x^2
for nu in range(2, nu + 1): # initialization takes care of order 1, so iterate from order 2
    q = 0
    for mu in range(1, nu): # Terms come from the previous derivative, so there are nu - 1 of them here.
        p = numers.popleft() # c = nu - mu/2
        numers.append(denom * p.deriv() + (nu - mu/2 - 1) * poly([0, 2]) * p - q)
        q = p
    numers.append(-q)

These are functions in $x$, but we can substitute to easily make them functions of $\cos(\theta)$, and then we can multipy by appropriate powers of $\sin(\theta)$ to put them all in the numerator of a single fraction.

In [23]:
exprs = deque()
for mu,p in enumerate(numers):
    expr = 0
    for r,c in enumerate(p.coef):
        expr += c * sp.cos(th)**r
    expr *= sp.sin(th)**mu
    exprs.append(expr)

In [24]:
print(exprs)

deque([-24.0*cos(th)**4 - 72.0*cos(th)**2 - 9.0, (50.0*cos(th)**3 + 55.0*cos(th))*sin(th), (-35.0*cos(th)**2 - 10.0)*sin(th)**2, 10.0*sin(th)**3*cos(th), -1.0*sin(th)**4])


Now we note that these are multiplying subsequent orders of $y^{(\mu)}(\theta)$, so the full numerator looks like:

$$a(\theta)y'(\theta) + b(\theta)y''(\theta) + ... z(\theta)y^{(\nu)}(\theta)$$

This situation will mean product rules as we take derivatives. Mildly gross, but like the pyramid scheme from before, each $y^{(\mu)}(\theta)$ depends only on two terms from the previous expression, so we can evaluate subsequent numerators actually pretty easily.

In [ ]:
for mu in range(2, nu + 1): # initialization takes care of order 1, so iterate from order 2
	q = 0
	for term in range(1, mu): # Terms come from the previous derivative, so there are mu - 1 of them here.
		p = numers.popleft() # c = mu - term/2
		numers.append(denom * p.deriv() + (mu - term/2 - 1) * poly([0, 2]) * p - q)
		q = p
	numers.append(-q)